# Answer Questions from Documents

Build a working Q&A system on your PDFs in under 3 minutes.


In [ ]:
%pip install -qU pixeltable sentence-transformers openai spacy tiktoken

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os, getpass
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')


In [ ]:
import pixeltable as pxt
from pixeltable.functions import openai
from pixeltable.functions.huggingface import sentence_transformer
from pixeltable.iterators import DocumentSplitter

In [ ]:
# Step 1: Store documents
pxt.create_dir('rag', if_exists='ignore')
docs = pxt.create_table('rag.docs', {'doc': pxt.Document}, if_exists='ignore')
docs.insert([{'doc': 'https://raw.githubusercontent.com/pixeltable/pixeltable/release/docs/resources/rag-demo/Zacks-Nvidia-Report.pdf'}])

Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/anushas-pxt/.pixeltable/pgdata
Created directory 'rag'.
Created table 'docs'.
Inserting rows into `docs`: 1 rows [00:00, 385.65 rows/s]
Inserted 1 row with 0 errors.


Error: limit/overlap requires the "token_limit" or "char_limit" separator

In [ ]:
# Step 2: Split into chunks with embedding index
chunks = pxt.create_view('rag.chunks', docs, if_exists='ignore',
    iterator=DocumentSplitter.create(document=docs.doc, separators='token_limit', limit=300))
chunks.add_embedding_index('text', if_exists='ignore',
    string_embed=sentence_transformer.using(model_id='intfloat/e5-large-v2'))

In [ ]:
# Step 3: Create retrieval query
@pxt.query
def get_context(question: str):
    sim = chunks.text.similarity(question)
    return chunks.order_by(sim, asc=False).select(chunks.text).limit(5)

In [ ]:
# Step 4: Create Q&A table with LLM
qa = pxt.create_table('rag.qa', {'question': pxt.String}, if_exists='ignore')
qa.add_computed_column(context=get_context(qa.question), if_exists='ignore')

@pxt.udf
def create_prompt(context: list[dict], question: str) -> str:
    passages = '\n\n'.join(item['text'] for item in context)
    return f'Context:\n{passages}\n\nQuestion: {question}'

qa.add_computed_column(prompt=create_prompt(qa.context, qa.question), if_exists='ignore')
qa.add_computed_column(if_exists='ignore',
    answer=openai.chat_completions(
        model='gpt-4o-mini',
        messages=[{'role': 'user', 'content': qa.prompt}]
    ).choices[0].message.content
)

In [ ]:
# Ask questions and get answers
qa.insert([{'question': 'What is the expected EPS for Nvidia?'}])

In [ ]:
# View results
qa.select(qa.question, qa.answer).head()

**What's Happening:**
- Documents → chunks (sentence-level splits)
- Chunks → embeddings (automatic indexing)
- Questions → similar chunks (vector search)
- Chunks + question → LLM → answer

**Next:** `let-ai-search-web-for-answers.ipynb` • `build-chatbot-with-memory.ipynb`
